# Bloomberg BQuant Spotlight Webinar Series: Adjusting Equity Factor Weights Based on Economic Regime
This is a companion notebook to the [Adjusting Equity Factor Weights Based on Economic Regime](https://blinks.bloomberg.com/screens/PLYR%20VOD%20334785689) webinar.

In [ ]:
import bql
import pandas as pd
from collections import OrderedDict

from bqplot import *
import bqviz as bqv

bq = bql.Service()

In [ ]:
eco_universe = bq.univ.members('WesternEurope',type='Region')
eco_field = bq.data.gdp(transform='PCT',pop='YoY')
eco_field = bq.func.dropna(bq.func.if_(eco_field > 4,'Expansion',bq.func.if_(eco_field < 2,'Contraction','Neutral')),remove_id='true')
eco_field_dict = {'Regime':eco_field}

request = bql.Request(eco_universe,eco_field_dict)
response = bq.execute(request)

eco_df = bql.combined_df(response)
eco_df.head()

In [ ]:
eco_df = eco_df.reset_index()
eco_df = eco_df.rename(columns={'ID':'Country Code'})

eco_df.head()

In [ ]:
bar_chart = bqv.BarPlot(pd.DataFrame(eco_df['Regime'].value_counts()),y_min=0)
bar_chart.show()

In [ ]:
def winsorize(factor,limit):
    lower_bound = -1 * limit
    upper_bound = limit
    return bq.func.if_(factor >= upper_bound,upper_bound,
                      bq.func.if_(factor <= lower_bound, lower_bound,factor))

In [ ]:
params = {'fill':'PREV','Currency':'EUR'}

In [ ]:
factor_model = OrderedDict()

factor_model = {
    
    "Descriptive":{
        "Name":{
            "expression":bq.data.name()
        },
        
        "Country":{
            "expression":bq.data.country_full_name()
        },
        
        "Country Code":{
            "expression":bq.data.country_iso()
        },
        
        "Sector":{
            "expression":bq.data.gics_sector_name()
        }
    },
    
    "Growth Model":{
        "[G] Sales Growth":{
            "expression":winsorize(bq.func.groupzscore(bq.data.sales_growth()),3),
            "weights":0.6
        },
        
        "[G] EPS Growth":{
            "expression":winsorize(bq.func.groupzscore(bq.data.eps_growth()),3),
            "weights":0.4
        }
    },
    
    "Value Model":{
        "[V] FCF Yield":{
            "expression":winsorize(bq.func.groupzscore(bq.data.free_cash_flow_yield()),3),
            "weights":0.55
        },
        
        "[V] Earnings Yield":{
            "expression":winsorize(bq.func.groupzscore(bq.data.is_eps()/bq.data.px_last()),3),
            "weights":0.45
        }
    }
}

In [ ]:
equity_universe = bq.univ.members('BE500 Index')

In [ ]:
request_dict = OrderedDict()

for category, individual_field in factor_model.items():
    for individual_field, individual_factor_setting in individual_field.items():
        request_dict[individual_field] = individual_factor_setting['expression']

In [ ]:
request = bql.Request(equity_universe,request_dict,with_params=params)
response = bq.execute(request)
output_df = pd.concat([r.df() for r in response],axis=1)[list(request_dict.keys())]
output_df.head()

In [ ]:
output_df = output_df.fillna(value=0)

In [ ]:
output_df['[G] Score'] = 0
output_df['[V] Score'] = 0
for category, individual_field in factor_model.items():
    for individual_field, individual_factor_setting in individual_field.items():
        if category == 'Value Model':
            output_df['[V] Score'] = output_df['[V] Score'] + output_df[individual_field] * individual_factor_setting['weights']
        elif category == 'Growth Model':
            output_df['[G] Score'] = output_df['[G] Score'] + output_df[individual_field] * individual_factor_setting['weights']
            
output_df.head()

In [ ]:
output_df = output_df.reset_index()
output_df = output_df.merge(eco_df,on='Country Code')
output_df.head()

In [ ]:
output_df.loc[(output_df['Regime'] == 'Contraction'),'Total Score'] = output_df['[G] Score'] * 0.6 + output_df['[V] Score'] * 0.4
output_df.loc[(output_df['Regime'] == 'Expansion'),'Total Score'] = output_df['[G] Score'] * 0.4 + output_df['[V] Score'] * 0.6
output_df.loc[(output_df['Regime'] == 'Neutral'),'Total Score'] = output_df['[G] Score'] * 0.5 + output_df['[V] Score'] * 0.5
output_df.head()

In [ ]:
output_df = output_df.round(3)
output_df = output_df.sort_values('Total Score',ascending=False)
output_df.head()

In [ ]:
output_df

In [ ]:
top_df = output_df[output_df['Total Score'].rank(ascending=False) <= 20]

In [ ]:
sc_x = LinearScale()
sc_y = LinearScale()
c_sc = OrdinalColorScale()

my_chart = Scatter(x = top_df['[G] Score'],
                  y = top_df['[V] Score'],
                  color = top_df['Regime'],
                  names = top_df['ID'],
                  display_names = False,
                  scales = {'x':sc_x,'y':sc_y,'color':c_sc},
                  tooltip = Tooltip(fields=['name','x','y'],labels=['Regime','[G] Score','[V] Score']))

ax_x = Axis(label='[G] Score',scale=sc_x)
ax_y = Axis(label='[V] Score',scale=sc_y,orientation='vertical')
ax_c = ColorAxis(label='Regime',scale=c_sc,side='right')

Figure(axes=[ax_x,ax_y,ax_c],marks=[my_chart],fig_margin={'top':10,'bottom':60,'left':60,'right':150})